In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
from numpy import genfromtxt
import scipy.misc
import matplotlib.pyplot as plt
import sklearn.datasets
import theano
import theano.tensor as T
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.layers import advanced_activations
from keras.layers.normalization import BatchNormalization
from keras.optimizers import *
from keras.layers.advanced_activations import *
from keras.regularizers import l2
import csv
import pickle
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.externals import joblib


In [ ]:
IMAGE_WIDTH = 96
IMAGE_HIGHT = 96
CHANNELS_NUM = 1
DATASET_SIZE = 7049
CLASSES_NUM = 6*2
EPOCHS_NUM = 500
BATCH_SIZE = 16
LEARNING_RATE = 0.005
FTRAIN = 'training.csv'
FTEST = 'test.csv'
FLOOKUP = 'CSVs/facial_points_IdLookupTable.csv'

In [2]:
def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=10)

       
def make_submission(test_labels):
    test_labels *= 48.0
    test_labels += 48.0
    test_labels = test_labels.clip(0, 96)

    lookup_table = pd.read_csv(FLOOKUP)
    values = []

    cols = joblib.load('cols.pkl')

    for index, row in lookup_table.iterrows():
        values.append((
            row['RowId'],
            test_labels[row.ImageId - 1][np.where(cols == row.FeatureName)[0][0]],
            ))
    submission = pd.DataFrame(values, columns=('RowId', 'Location'))
    submission.to_csv('CSVs/submission.csv', index=False)
    
def load(test=False, cols=None):
    fname = FTEST if test else FTRAIN
    df = read_csv(fname)  
    cols_l = df.columns[:-1]
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    if cols:  
        df = df[list(cols) + ['Image']]

    print(df.count())  
    df = df.dropna()
    X = np.vstack(df['Image'].values)  
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
        joblib.dump(cols_l, 'cols.pkl', compress=3)
    else:
        y = None
    return X, y

In [ ]:
cols=['left_eye_center_x',
      'left_eye_center_y',
      'right_eye_center_x',
      'right_eye_center_y',
      'nose_tip_x',
      'nose_tip_y',
      'mouth_center_bottom_lip_x',
      'mouth_center_bottom_lip_y']
X, y = load(cols= cols)
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

In [ ]:
def build_model():
    model = Sequential()
    
    model.add(Convolution2D(16, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same',
                            input_shape=(CHANNELS_NUM,IMAGE_HIGHT, IMAGE_WIDTH)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))
    
    model.add(Convolution2D(32, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))
    
    model.add(Convolution2D(16, 3, 3, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))
    
    model.add(Convolution2D(8, 1, 1, init = 'he_normal', W_regularizer = l2(1e-4), border_mode='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(64, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))
    
    model.add(Dense(32, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.15))
    model.add(Dropout(0.3))

    model.add(Dense(CLASSES_NUM, init = 'he_normal', W_regularizer = l2(1e-4)))
    model.add(Activation('linear'))
    return model

In [ ]:
inzeption = build_model()
inzeption.summary()

In [ ]:
adam = Adam(lr=LEARNING_RATE, decay=0.001)
inzeption.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
try:
    inzeption.fit(X.reshape(-1, 1,96, 96), y, batch_size=16, nb_epoch=100, validation_split= 0.1,verbose=1, shuffle=True)
except KeyboardInterrupt:
    pass

In [ ]:
inzeption.save('my_3nd_model.h5')

In [ ]:
X, _ = load(test=True)
X=X.reshape((-1,96,96,1))
y_pred = inzeption.predict(X)

fig = plt.figure(figsize=(6, 6))
fig.subplots_adjust(
left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(16):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    plot_sample(X[i], y_pred[i], ax)

plt.show()

In [ ]:
tst = load(test=True)[0].reshape(-1, 96, 96, 1)
y_pred1 = inzeption.predict(tst)
print y_pred1.shape, tst.shape
make_submission(y_pred1)